## Assignment

Apply PCA on 50 Startups Dataset.

Ensure you perform deployment of the same

In [61]:
import numpy as np
import pandas as pd

In [62]:
data = pd.read_csv('50_Startups.csv')

In [63]:
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [65]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
fState = ohe.fit_transform(np.array(data['State']).reshape(-1,1))
#fState

In [36]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#data.State = le.fit_transform(data.State)

In [67]:
features = np.concatenate([fState, np.array(data.iloc[:,[0,1,2]])] , axis = 1)

#features = data.iloc[:,:-1].values
label = data.iloc[:, -1].values

In [10]:
#PCA
# Guidelines:
# 1. Standardization(StandardScaler) is mandatory.
# 2. Hyperparameter for PCA (n_components <= no of features)
# 3. n_components can be judged using :
#          count based on variance(component value should be greater than or 
#           equal to 75%)

In [68]:
# 1. Standardization(StandardScaler) is mandatory.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features = sc.fit_transform(features)

In [23]:
#features

In [69]:
# using this part for explorations

# 2. Initialize the PCA and detect the number of principal components
from sklearn.decomposition import PCA
principalComponents = PCA(n_components=6) #Here n_components = n_features
principalComponents.fit(features,label)

PCA(n_components=6)

In [70]:
#Identify the ideal n_components
principalComponents.explained_variance_ratio_

array([3.26596913e-01, 2.51852089e-01, 2.16730846e-01, 1.68048197e-01,
       3.67719551e-02, 2.60611749e-32])

In [ ]:
#Since none of the components are greater than or equal to 0.75, we will choose
# n_components = 1
#Guideline to ensure u get a high quality model
#e.g. array([0.8988888, 0.82264637, 0.727833, 0.108765]) --> n_comp= 2

In [71]:
# using this part for finalize the model

#Step3: Apply PCA with correct n_components
from sklearn.decomposition import PCA
principalComponentsFinal = PCA(n_components=1) #Here n_components = n_features  / here we have trained PCA
principalComponentsFinal.fit(features,label)

PCA(n_components=1)

In [72]:
principalComponentsFinal.explained_variance_ratio_

array([0.32659691])

In [73]:
#Step4: Transform your feature set /  usind the trained PCA, we are recreating our features
finalFeatures = principalComponentsFinal.transform(features)

### Exploration for KNeighborsRegressor to find best train_test_split

In [82]:
SL = 0.1
CL = 1 - SL

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

for i in range(1,3000):
    X_train,X_test,y_train,y_test = train_test_split(finalFeatures,label,test_size=0.1,random_state=i)
    knr = KNeighborsRegressor(n_neighbors=7)
    #model = BaggingRegressor(n_estimators=5, #No of weak learners
                         #base_estimator=knr) #The algo to be used for learning
    knr.fit(X_train,y_train)
    train_score = knr.score(X_train,y_train)
    test_score = knr.score(X_test,y_test)
    
    if test_score > train_score and test_score > CL:
        print("Test Score {}, Train Score {}, RandomSeed {}".format(test_score,train_score,i))

Test Score 0.9337211099023445, Train Score 0.7065275688397826, RandomSeed 17
Test Score 0.9005106670057083, Train Score 0.7036866689631882, RandomSeed 60
Test Score 0.9311720105842937, Train Score 0.6746590804109218, RandomSeed 306
Test Score 0.9483313736316629, Train Score 0.712237335451444, RandomSeed 325
Test Score 0.9243949173640865, Train Score 0.7025830292083617, RandomSeed 328
Test Score 0.9234323863128409, Train Score 0.6701512137225252, RandomSeed 378
Test Score 0.9317378379528075, Train Score 0.6912087249252477, RandomSeed 525
Test Score 0.9029943478247453, Train Score 0.7062550934085838, RandomSeed 529
Test Score 0.9524718142323688, Train Score 0.710419298738937, RandomSeed 542
Test Score 0.9149938562724854, Train Score 0.6854207630567628, RandomSeed 618
Test Score 0.9308653686349264, Train Score 0.6844836408432617, RandomSeed 623
Test Score 0.9461468231485978, Train Score 0.682818083844491, RandomSeed 687
Test Score 0.9628465728204758, Train Score 0.7133061907637538, Random

In [ ]:
# we can use thie=s train test split:   Test Score 0.9742816856637477, Train Score 0.708333194919203, RandomSeed 1228

### Create final model

In [83]:
from sklearn.model_selection import train_test_split
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

X_train,X_test,y_train,y_test = train_test_split(finalFeatures,  # finalFeatures is our principal component and we use it in train_test_split 
                                                label,
                                                test_size=0.1,
                                                random_state=1228)

knn = KNeighborsRegressor()
knn.fit(X_train,y_train)

KNeighborsRegressor()

In [84]:
knn.score(X_train,y_train)

0.7286468979464145

In [85]:
knn.score(X_test,y_test)

0.9421062520475576

In [94]:
## Deployment

rndSpend = float(input("Enter R&D Spending: "))
adminSpend =float(input("Enter Administration Spending: "))
marketSpend = float(input("Enter Marketing Spending: "))
state = input("Where is your company located? (New York, California, or Florida):  ")
if state == "New York":
    fState = [0,0,1]
elif state == "Florida":
    fState = [0,1,0]
else:
    fState = [1,0,0]

#FeatureSet
featureSet = np.concatenate( [[fState] , np.array([[rndSpend,adminSpend,marketSpend]])], axis = 1)

#Standardization
featureStandardizedSet = sc.transform(featureSet)

#transform features using PCA
principalCFeatures = principalComponentsFinal.transform(featureStandardizedSet)

#Predict
print(knn.predict(principalCFeatures))

[91881.83]


In [95]:
#Pickle Code
import pickle
#Deploy StandardScaler Object
pickle.dump(sc,open("ScalerFor50startups.scale",'wb'))
#Deploy PCA object
pickle.dump(principalComponentsFinal,open('PCA.pca','wb'))
#Deploy Model
pickle.dump(knn,open('Model50startups.pkl','wb'))